In [1]:
import glob
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

/home/keller/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
import torch
import tiktoken

In [3]:
from datasets import load_dataset
dset = load_dataset("HuggingFaceFW/fineweb", name='sample-10BT', split="train", cache_dir='/data-4/keller/cache/')

Resolving data files:   0%|          | 0/23781 [00:00<?, ?it/s]

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading dataset shards:   0%|          | 0/102 [00:00<?, ?it/s]

In [177]:
for i in tqdm(range(1000000)):
    t = dset[i]['text']
    with open('texts/%d.txt' % i, 'w') as f:
        f.write(t)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [00:54<00:00, 9148.66it/s]


In [226]:
import bpeasy

In [247]:
class MyIter:
    def __init__(self, n):
        self.idx = 0
        self.files = glob.glob('texts/*.txt')[:n]
    def __iter__(self):
        while True:
            with open(self.files[self.idx]) as f:
                s = f.read()
                yield s
            self.idx += 1
            if self.idx == len(self.files):
                return

In [ ]:
%%time

# n_examples = 10000
# n_examples = 200000
n_examples = 1000000

iterator = iter(MyIter(n_examples))

enc = tiktoken.get_encoding("gpt2")
regex_pattern = enc._pat_str

# n_vocab = 37000
n_vocab = 10000

# returns the vocab (dict[bytes, int])
vocab = bpeasy.train_bpe(
    iterator,
    enc._pat_str,
    1000000,
    n_vocab,
)

# Sort the vocab by rank
sorted_vocab = sorted(list(vocab.items()), key=lambda x: x[1])
encoder = tiktoken.Encoding(
            name='fineweb30k',
            pat_str=enc._pat_str,
            mergeable_ranks=dict(sorted_vocab),
            special_tokens={'<|endoftext|>': n_vocab},
        )

In [ ]:
enc = tiktoken.get_encoding("gpt2")
xx = []
yy = []
for i in tqdm(range(10000)):
    t = dset[int(1e6)+i]['text']
    # t = dset[i]['text']
    xx.append(len(encoder.encode_ordinary(t)))
    yy.append(len(enc.encode_ordinary(t)))

print(sum(xx) / len(xx))
print(sum(yy) / len(yy))
print(sum(xx) / sum(yy))

In [ ]:
# bpeasy.save_vocab_to_tiktoken(dict(sorted_vocab), "bpeasy_fineweb_n1000k_v37k.txt", special_tokens=["<|endoftext|>"])
bpeasy.save_vocab_to_tiktoken(dict(sorted_vocab), "bpeasy_fineweb_n1000k_v10k.txt", special_tokens=["<|endoftext|>"])

In [392]:
# sorted_ranks = tiktoken.load.load_tiktoken_bpe('./bpeasy_fineweb_n1000k_v37k.txt')
sorted_ranks = tiktoken.load.load_tiktoken_bpe('./bpeasy_fineweb_n1000k_v10k.txt')
# sorted_ranks = tiktoken.load.load_tiktoken_bpe('./bpeasy_fineweb_n200k_v10k.txt')
# sorted_ranks = tiktoken.load.load_tiktoken_bpe('/data-4/keller/notebooks/bpeasy_fineweb_n200k_v10k.txt')
enc2 = tiktoken.Encoding(
    name='fineweb_n1000k_v37k',
    pat_str=enc._pat_str,
    mergeable_ranks=sorted_ranks,
    special_tokens={'<|endoftext|>': len(sorted_ranks)},
)

enc = tiktoken.get_encoding("gpt2")
xx = []
yy = []
for i in tqdm(range(50000)):
    t = dset[int(1e6)+i]['text']
    xx.append(len(enc2.encode_ordinary(t)))
    yy.append(len(enc.encode_ordinary(t)))

print(sum(xx) / len(xx))
print(sum(yy) / len(yy))
print(sum(xx) / sum(yy))